# import

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
import gender_guesser.detector as gender
from sklearn import metrics

from sklearn.metrics import classification_report

# load data

In [ ]:
data1 = pd.read_csv('cleandata.csv')
y = data1.account_type.values.tolist()